In [1]:
import sys
import numpy as np
from numpy import percentile as per
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Save Model Using Pickle
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
# call csv to prepare for DB call

airlines_2017 = pd.read_csv('combined_2017_csv.csv')
airlines_2017

C:\Users\supra\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,ORIGIN_AIRPORT_ID,...,CANCELLATION_CODE,DIVERTED,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 37
0,2017,2,4,10,1,2017-04-10,DL,19790,DL,12478,...,NaN,0.0,203.0,1521.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,2,4,10,1,2017-04-10,DL,19790,DL,14685,...,NaN,0.0,43.0,214.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,2,4,10,1,2017-04-10,DL,19790,DL,10397,...,NaN,0.0,300.0,2182.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,2,4,10,1,2017-04-10,DL,19790,DL,14771,...,NaN,0.0,298.0,2586.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,2,4,10,1,2017-04-10,DL,19790,DL,10397,...,NaN,0.0,35.0,214.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5674616,2017,3,9,6,3,2017-09-06,WN,19393,WN,13796,...,NaN,0.0,65.0,407.0,NaN,NaN,NaN,NaN,NaN,NaN
5674617,2017,3,9,6,3,2017-09-06,WN,19393,WN,13796,...,NaN,0.0,67.0,407.0,NaN,NaN,NaN,NaN,NaN,NaN
5674618,2017,3,9,6,3,2017-09-06,WN,19393,WN,13796,...,NaN,0.0,56.0,337.0,NaN,NaN,NaN,NaN,NaN,NaN
5674619,2017,3,9,6,3,2017-09-06,WN,19393,WN,13796,...,NaN,0.0,57.0,337.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# checking columns
airlines_2017.columns

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID',
       'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST',
       'DEST_CITY_NAME', 'DEST_STATE_ABR', 'DEST_STATE_NM', 'DEP_DELAY',
       'DEP_DELAY_NEW', 'DEP_DEL15', 'DEP_DELAY_GROUP', 'CANCELLED',
       'CANCELLATION_CODE', 'DIVERTED', 'AIR_TIME', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY', 'Unnamed: 37'],
      dtype='object')

In [4]:
# creating groups for carrier delay average
airlines_2017_df =airlines_2017.groupby(['MONTH','DAY_OF_MONTH','OP_UNIQUE_CARRIER'], as_index=False)['CARRIER_DELAY'].mean()
airlines_2017_df

,MONTH,DAY_OF_MONTH,OP_UNIQUE_CARRIER,CARRIER_DELAY
0,1,1,AA,29.336538
1,1,1,AS,6.929889
2,1,1,B6,24.343750
3,1,1,DL,26.233227
4,1,1,EV,52.959641
...,...,...,...,...
4375,12,31,NK,29.750000
4376,12,31,OO,39.115108
4377,12,31,UA,20.631285
4378,12,31,VX,13.432432


In [5]:
#assigning x and y

X = airlines_2017_df[["MONTH","DAY_OF_MONTH"]]
y = airlines_2017_df["CARRIER_DELAY"]
# print("Shape: ", X.shape, y.shape)


# X = ['MONTH','DAY_OF_MONTH']# add after changing to 01s 'OP_UNIQUE_CARRIER'
# y = ['CARRIER_DELAY']



In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 47, test_size = 0.20)

In [7]:
model = LinearRegression()

In [8]:
model.fit(X_train, y_train)



LinearRegression()

In [9]:
# Use our model to make predictions
predicted = model.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

Mean Squared Error (MSE): 133.66442516785204
R-squared (R2 ): -0.0013830281183697668


In [10]:
model.score(X_test, y_test)

-0.0013830281183697668

In [11]:
airlines_2017_df =airlines_2017.groupby(['MONTH','DAY_OF_MONTH','OP_UNIQUE_CARRIER'], as_index=False)['CARRIER_DELAY'].mean()
airlines_2017_df

,MONTH,DAY_OF_MONTH,OP_UNIQUE_CARRIER,CARRIER_DELAY
0,1,1,AA,29.336538
1,1,1,AS,6.929889
2,1,1,B6,24.343750
3,1,1,DL,26.233227
4,1,1,EV,52.959641
...,...,...,...,...
4375,12,31,NK,29.750000
4376,12,31,OO,39.115108
4377,12,31,UA,20.631285
4378,12,31,VX,13.432432


In [12]:
# ONE hot reformat
# Step 1: Label-encode data set
op_reformat = airlines_2017_df.OP_UNIQUE_CARRIER
label_encoder = LabelEncoder()
label_encoder.fit(op_reformat)
encoded_y = label_encoder.transform(op_reformat)
encoded_y

array([ 0,  1,  2, ...,  9, 10, 11])

In [13]:
# step 2: check for values and DF 
op_reformat.unique()

array(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'NK', 'OO', 'UA', 'VX',
       'WN'], dtype=object)

In [14]:
hacha_df = pd.DataFrame(encoded_y,op_reformat)
hacha_df.head(13)

,0
OP_UNIQUE_CARRIER,
AA,0
AS,1
B6,2
DL,3
EV,4
F9,5
HA,6
NK,7
OO,8


In [15]:
# step3 : need to create new column for labels to use on machine learning

conditions = [(airlines_2017_df['OP_UNIQUE_CARRIER']== 'AA'),#1
             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'AS'),#2
             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'B6'),#3
             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'DL'),#4
            (airlines_2017_df['OP_UNIQUE_CARRIER']== 'EV'),#5
             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'F9'),#6
             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'HA'),#7
             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'NK'),#8
             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'OO'),#9
             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'UA'),#10
             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'VX'),#11
             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'WN')]#12

values = ['0','1','2','3','4','5','6','7','8','9','10','11']

airlines_2017_df['labenc_OP_UNIQUE_CARRIER'] = np.select(conditions,values)
airlines_2017_df

,MONTH,DAY_OF_MONTH,OP_UNIQUE_CARRIER,CARRIER_DELAY,labenc_OP_UNIQUE_CARRIER
0,1,1,AA,29.336538,0
1,1,1,AS,6.929889,1
2,1,1,B6,24.343750,2
3,1,1,DL,26.233227,3
4,1,1,EV,52.959641,4
...,...,...,...,...,...
4375,12,31,NK,29.750000,7
4376,12,31,OO,39.115108,8
4377,12,31,UA,20.631285,9
4378,12,31,VX,13.432432,10


In [16]:
# assign x and y to values
X2 = airlines_2017_df[["MONTH","DAY_OF_MONTH","labenc_OP_UNIQUE_CARRIER"]]
y2 = airlines_2017_df["CARRIER_DELAY"]

In [17]:
# training module

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state = 47, test_size = 0.20)

In [18]:
# model = LinearRegression()
model.fit(X2_train, y2_train)

LinearRegression()

In [19]:
# Use our model to make predictions
predicted2 = model.predict(X2_test)

# Score the predictions with mse and r2
mse2 = mean_squared_error(y2_test, predicted)
r22 = r2_score(y2_test, predicted2)

print(f"Mean Squared Error (MSE): {mse2}")
print(f"R-squared (R2 ): {r22}")

Mean Squared Error (MSE): 133.66442516785204
R-squared (R2 ): 0.01911841828176264


In [20]:

# conditions = [(airlines_2017_df['OP_UNIQUE_CARRIER']== 'AA'),#1
#              (airlines_2017_df['OP_UNIQUE_CARRIER']== 'AS'),#2
#              (airlines_2017_df['OP_UNIQUE_CARRIER']== 'B6'),#3
#              (airlines_2017_df['OP_UNIQUE_CARRIER']== 'DL'),#4
#             (airlines_2017_df['OP_UNIQUE_CARRIER']== 'EV'),#5
#              (airlines_2017_df['OP_UNIQUE_CARRIER']== 'F9'),#6
#              (airlines_2017_df['OP_UNIQUE_CARRIER']== 'HA'),#7
#              (airlines_2017_df['OP_UNIQUE_CARRIER']== 'NK'),#8
#              (airlines_2017_df['OP_UNIQUE_CARRIER']== 'OO'),#9
#              (airlines_2017_df['OP_UNIQUE_CARRIER']== 'UA'),#10
#              (airlines_2017_df['OP_UNIQUE_CARRIER']== 'VX'),#11
#              (airlines_2017_df['OP_UNIQUE_CARRIER']== 'WN')]#12

# values = ['0','1','2','3','4','5','6','7','8','9','10','11']

# airlines_2017['labenc_OP_UNIQUE_CARRIER'] = np.select(conditions,values)
# airlines_2017.head(13)

In [21]:

# checking data sets for delays
airlines_dest_df =airlines_2017.groupby(['MONTH','DAY_OF_MONTH','DEST_AIRPORT_ID'], as_index=False)['CARRIER_DELAY'].mean()
airlines_dest_df

,MONTH,DAY_OF_MONTH,DEST_AIRPORT_ID,CARRIER_DELAY
0,1,1,10135,10.000000
1,1,1,10136,NaN
2,1,1,10140,8.375000
3,1,1,10141,NaN
4,1,1,10146,23.000000
...,...,...,...,...
105651,12,31,15624,16.666667
105652,12,31,15841,NaN
105653,12,31,15919,NaN
105654,12,31,15991,NaN


In [22]:
# checking data sets for delays
airlines_dest_df =airlines_2017.groupby(['MONTH','DAY_OF_MONTH','ORIGIN'], as_index=False)['CARRIER_DELAY'].mean()
airlines_dest_df

,MONTH,DAY_OF_MONTH,ORIGIN,CARRIER_DELAY
0,1,1,ABE,NaN
1,1,1,ABI,NaN
2,1,1,ABQ,4.0
3,1,1,ABR,NaN
4,1,1,ABY,0.0
...,...,...,...,...
105575,12,31,VPS,0.0
105576,12,31,WRG,NaN
105577,12,31,XNA,NaN
105578,12,31,YAK,NaN


In [23]:
# checking data sets for delays
airlines_dest_df =airlines_2017.groupby(['MONTH','DAY_OF_MONTH','ORIGIN_AIRPORT_ID'], as_index=False)['CARRIER_DELAY'].mean()
airlines_dest_df

,MONTH,DAY_OF_MONTH,ORIGIN_AIRPORT_ID,CARRIER_DELAY
0,1,1,10135,NaN
1,1,1,10136,NaN
2,1,1,10140,4.0
3,1,1,10141,NaN
4,1,1,10146,0.0
...,...,...,...,...
105575,12,31,15624,0.0
105576,12,31,15841,NaN
105577,12,31,15919,NaN
105578,12,31,15991,NaN


In [24]:
# checking data sets for delays
airlines_dow_df =airlines_2017.groupby(['MONTH','DAY_OF_MONTH','DAY_OF_WEEK','OP_UNIQUE_CARRIER'], as_index=False)['CARRIER_DELAY'].mean()
airlines_dow_df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,CARRIER_DELAY
0,1,1,7,AA,29.336538
1,1,1,7,AS,6.929889
2,1,1,7,B6,24.343750
3,1,1,7,DL,26.233227
4,1,1,7,EV,52.959641
...,...,...,...,...,...
4375,12,31,7,NK,29.750000
4376,12,31,7,OO,39.115108
4377,12,31,7,UA,20.631285
4378,12,31,7,VX,13.432432


In [25]:
# check for machine learning with day of week as well
op_reformat_dow = airlines_dow_df.OP_UNIQUE_CARRIER
label_encoder_dow = LabelEncoder()
label_encoder.fit(op_reformat_dow)
encoded_y_dow = label_encoder.transform(op_reformat_dow)
encoded_y_dow

array([ 0,  1,  2, ...,  9, 10, 11])

In [26]:
hacha_df_dow = pd.DataFrame(encoded_y_dow,op_reformat_dow)
hacha_df_dow.head(13)

,0
OP_UNIQUE_CARRIER,
AA,0
AS,1
B6,2
DL,3
EV,4
F9,5
HA,6
NK,7
OO,8


In [27]:
conditions_dow = [(airlines_dow_df['OP_UNIQUE_CARRIER']== 'AA'),#1
             (airlines_dow_df['OP_UNIQUE_CARRIER']== 'AS'),#2
             (airlines_dow_df['OP_UNIQUE_CARRIER']== 'B6'),#3
             (airlines_dow_df['OP_UNIQUE_CARRIER']== 'DL'),#4
            (airlines_dow_df['OP_UNIQUE_CARRIER']== 'EV'),#5
             (airlines_dow_df['OP_UNIQUE_CARRIER']== 'F9'),#6
             (airlines_dow_df['OP_UNIQUE_CARRIER']== 'HA'),#7
             (airlines_dow_df['OP_UNIQUE_CARRIER']== 'NK'),#8
             (airlines_dow_df['OP_UNIQUE_CARRIER']== 'OO'),#9
             (airlines_dow_df['OP_UNIQUE_CARRIER']== 'UA'),#10
             (airlines_dow_df['OP_UNIQUE_CARRIER']== 'VX'),#11
             (airlines_dow_df['OP_UNIQUE_CARRIER']== 'WN')]#12

values_dow = ['0','1','2','3','4','5','6','7','8','9','10','11']

airlines_dow_df['labenc_OP_UNIQUE_CARRIER'] = np.select(conditions_dow,values_dow)
airlines_dow_df.head(13)

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,CARRIER_DELAY,labenc_OP_UNIQUE_CARRIER
0,1,1,7,AA,29.336538,0
1,1,1,7,AS,6.929889,1
2,1,1,7,B6,24.343750,2
3,1,1,7,DL,26.233227,3
4,1,1,7,EV,52.959641,4
5,1,1,7,F9,19.220588,5
6,1,1,7,HA,33.944444,6
7,1,1,7,NK,0.000000,7
8,1,1,7,OO,19.407051,8
9,1,1,7,UA,28.809756,9


In [29]:
X3 = airlines_dow_df[["MONTH","DAY_OF_MONTH",'DAY_OF_WEEK','labenc_OP_UNIQUE_CARRIER']]
y3 = airlines_dow_df["CARRIER_DELAY"]

In [30]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, random_state = 47, test_size = 0.20)

In [31]:
# model = LinearRegression()
model.fit(X3_train, y3_train)

LinearRegression()

In [33]:
# Use our model to make predictions
predicted3 = model.predict(X3_test)

# Score the predictions with mse and r2
mse3 = mean_squared_error(y3_test, predicted)
r23 = r2_score(y3_test, predicted3)

print(f"Mean Squared Error (MSE): {mse3}")
print(f"R-squared (R2 ): {r23}")

Mean Squared Error (MSE): 133.66442516785204
R-squared (R2 ): 0.037532125371789515


In [35]:
# op_reformat
# for label, original_class in zip(encoded_y, op_reformat):
#     print('Original Class: ' + str(original_class))
#     print('Encoded Label: ' + str(label))
#     print('-' * 20)

# from tensorflow.keras.utils import to_categorical

# # Step 2: One-hot encoding
# one_hot_op = to_categorical(encoded_y)
# one_hot_op 

In [36]:
# one_hot_df = pd.DataFrame(one_hot_op)
# one_hot_df.info()

In [34]:
# one_hot_df.columns = OneHotEncoder().get_feature_names()

# test_one['one_hot_op_1']= one_hot_df[0] +" "+ one_hot_df[1]#+" "+ one_hot_df[2]+" "+ one_hot_df[3]+" "+ one_hot_df[4]
# # # +" "+ one_hot_df["5"]+" "+ one_hot_df["6"]+" "+ one_hot_df["7"]+" "+ one_hot_df["8"]+" "+ one_hot_df["9"]+" "+ one_hot_df["10"]|
# # # +" "+ one_hot_df["11"]
# # print(df)

In [ ]:
airlines_2017_df =airlines_2017.groupby(['MONTH','DAY_OF_MONTH','OP_UNIQUE_CARRIER','DEST_AIRPORT_ID','ORIGIN_AIRPORT_ID'], as_index=False)['CARRIER_DELAY'].mean()
airlines_2017_df

In [25]:
# information in data set
    
print('Number of rows = ', airlines_2017.shape[0],'\n')            
print('Number of columns = ', airlines_2017.shape[1],'\n')           
print('Number of duplicated rows = ', airlines_2017.duplicated().sum(),'\n')            
print('Number of null values = ',airlines_2017.isna().sum().sum(),'\n')
            
print(airlines_2017.dtypes)

Number of rows =  5674621 

Number of columns =  38 

Number of duplicated rows =  82146 

Number of null values =  34908872 

YEAR                       int64
QUARTER                    int64
MONTH                      int64
DAY_OF_MONTH               int64
DAY_OF_WEEK                int64
FL_DATE                   object
OP_UNIQUE_CARRIER         object
OP_CARRIER_AIRLINE_ID      int64
OP_CARRIER                object
ORIGIN_AIRPORT_ID          int64
ORIGIN_AIRPORT_SEQ_ID      int64
ORIGIN_CITY_MARKET_ID      int64
ORIGIN                    object
ORIGIN_CITY_NAME          object
ORIGIN_STATE_ABR          object
ORIGIN_STATE_NM           object
DEST_AIRPORT_ID            int64
DEST_AIRPORT_SEQ_ID        int64
DEST_CITY_MARKET_ID        int64
DEST                      object
DEST_CITY_NAME            object
DEST_STATE_ABR            object
DEST_STATE_NM             object
DEP_DELAY                float64
DEP_DELAY_NEW            float64
DEP_DEL15                float64
DEP_DELAY_GROUP

In [ ]:
airlines_2017.hist(layout = (7,6),figsize=(15,20))
plt.show()

In [ ]:
# clean_airline_df_1 = airlines_2017[['YEAR', 'DEP_DELAY_GROUP']]
# clean_airline_df_1

In [ ]:
airlines_2017.columns


In [ ]:
al_2017_car = airlines_2017.groupby('YEAR')
al_2017_car.head()

In [ ]:
get_al_2017_car = al_2017_car.get_group('OP_UNIQUE_CARRIER')
get_al_2017_car

In [ ]:
# # 123 = airlines_2017.groupby(level=['FL_DATE','OP_UNIQUE_CARRIER']).sum()
# a2017_groupby = airlines_2017.groupby(['OP_UNIQUE_CARRIER','FL_DATE']).size().groupby('FL_DATE')#[['DEP_DELAY']].mean()

# # display(a2017_groupby)
# # with a2017_groupby.option_context('display.max_rows', None, 'display.max_columns', None): 
# #     print(a2017_groupby)

# a2017_groupby.head()

In [ ]:
grouped_single = airlines_2017.groupby('FL_DATE','OP_UNIQUE_CARRIER').agg({'DEP_DELAY': ['mean', 'min', 'max']})

grouped_single

In [ ]:
yqmdofweek = airlines_2017[['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK']]
yqmdofweek

In [ ]:
gb_airlines_2017_df = airlines_2017.groupby("OP_UNIQUE_CARRIER").mean()
gb_airlines_2017_df.head()

In [ ]:
clean_airline_df = gb_airlines_2017_df[['YEAR', 'DEP_DELAY','DEP_DELAY_NEW', 'DEP_DEL15']]
clean_airline_df